# Bachelor Programmierkurs 2016
In diesem Notebook sollen die grundlegenden Konzepte von RooFit vermittelt werden.

TODO:
    - s-Fits
    - Latex in Plots
    - RooFitResult.Print() klappt nicht ;(

In [ ]:
import ROOT

import os

from IPython.display import Image

import matplotlib.pyplot
%matplotlib inline

Erstelle ein Fitmodell, bestehend aus einer Signal- (Gaußverteilung) und einer Untergrundkomponente (Argus, sieht etwa [so](https://inspirehep.net/record/833350/files/figs_0140804-020.png) aus)

In [ ]:
# Observable
mass = ROOT.RooRealVar('mass', 'mass', 5000, 5500, 'MeV/#it{c}^{2}')

# Erstelle eine Signal-PDF, bestehend aus einer Gaußfunktion
sigMean = ROOT.RooRealVar('sigMean', 'sigMean', 5280, 5200, 5300)
sigWidth = ROOT.RooRealVar('sigWidth', 'sigWidth', 20, 1, 100)
sigGauss = ROOT.RooGaussian('sigGauss', 'sigGauss PDF', mass, sigMean, sigWidth)

# Erstelle eine Untergrund-PDF, bestehend aus einer Argusfunktion
bkgArgPar = ROOT.RooRealVar('bkgArgPar', 'bkgArgPar', -20, -100, -1)
bkgArgConst = ROOT.RooRealVar('bkgArgConst', 'bkgArgConst', 5291, 5200, 5400)
bkgArgus = ROOT.RooArgusBG('bkgArgus', 'bkgArgus PDF', mass, bkgArgConst, bkgArgPar)

# Kombiniere Signal + Untergrung PDF
nSig = ROOT.RooRealVar('nSig', '#signal events', 100, 200, 10000)
nBkg = ROOT.RooRealVar('nBkg', '#background events', 50, 200, 10000)
model = ROOT.RooAddPdf('model', 'complete fitmodel', ROOT.RooArgList(sigGauss, bkgArgus), 
                                                     ROOT.RooArgList(nSig, nBkg))
model.Print('V') 

In [ ]:
numDataPoints = 200

# Generiere ein ToyMC aus der gegebenen PDF
data = model.generate(ROOT.RooArgSet(mass), numDataPoints)

In [ ]:
plotPath = 'Plots'

if not os.path.exists(plotPath):
    os.makedirs(plotPath)

# Führe einen Fit an die Daten durch
fitResult = model.fitTo(data, ROOT.RooFit.Save(), ROOT.RooFit.Extended()) 

# Plotte die Datenpunkte und die PDF in ein Frame
massFrame = mass.frame()

data.plotOn(massFrame)
model.plotOn(massFrame)

# Das folgende funktioniert leider nicht :( ... Siehe https://root.cern.ch/phpBB3/viewtopic.php?t=7764
#### model.plotOn(massFrame, ROOT.RooFit.Components(ROOT.RooArgSet(bkgArgus), ROOT.RooFit.LineStyle(ROOT.kDashed)))

# Plotte die einzelnen Komponenten auf das Frame
argusArgSet = ROOT.RooArgSet(bkgArgus)
gaussArgSet = ROOT.RooArgSet(sigGauss)

model.plotOn(massFrame, ROOT.RooFit.Components(argusArgSet), 
                        ROOT.RooFit.LineColor(ROOT.kRed),
                        ROOT.RooFit.LineStyle(ROOT.kDashed))

model.plotOn(massFrame, ROOT.RooFit.Components(gaussArgSet), 
                        ROOT.RooFit.LineColor(ROOT.kGreen + 3),
                        ROOT.RooFit.LineStyle(ROOT.kDashed))


# Erzeuge ein TCanvas und plotte das Frame.
can = ROOT.TCanvas('can', 'can', 800, 600)

massFrame.Draw()

can.Draw()
can.SaveAs(os.path.join(plotPath, 'Simple_Massfit.pdf'))

In [ ]:
# Stelle das Model grafisch dar
import networkx as nx
import pydot
from IPython.display import Image

model.graphVizTree('SimpleModel.dot')
G = nx.drawing.nx_agraph.read_dot('SimpleModel.dot')
model_dot = nx.to_pydot(G)

model_png = model_dot.create_png(prog='dot')
Image(model_png)